# Factorizations and other fun

Based on the work by Andreas Noack

## Outline

* Factorizations
* Special matrix structures
* Generic linear algebra

Before we get started, let's set up a linear system and use LinearAlgebra to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra

In [2]:
A = rand(3,3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.6285190519865413
 0.9876967412478956
 1.8832519848049258

# Factorizations

**LU Factorizations**

In Julia we can perform an LU factorization

` PA = LU `

where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [3]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0      0.0
 0.478868  1.0      0.0
 0.300171  0.29519  1.0
U factor:
3×3 Matrix{Float64}:
 0.926838  0.659477  0.296938
 0.0       0.4014    0.325291
 0.0       0.0       0.207888

In [4]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

In [5]:
Alu.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [6]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0      0.0
 0.478868  1.0      0.0
 0.300171  0.29519  1.0

In [7]:
Alu.U

3×3 Matrix{Float64}:
 0.926838  0.659477  0.296938
 0.0       0.4014    0.325291
 0.0       0.0       0.207888

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [8]:
A\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant `A` using either `A` or the factorization object.

In [11]:
det(A) ≈ det(Alu)

StackOverflowError: StackOverflowError:

**QR factorizations**

In Julia, we can perform a QR factorization

` A = QR`

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [12]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.416893  -0.864792  -0.279884
 -0.261323  -0.180886   0.948151
 -0.870581   0.468418  -0.15058
R factor:
3×3 Matrix{Float64}:
 -1.06462  -0.955818  -0.55611
  0.0      -0.36856   -0.336282
  0.0       0.0        0.197109

Similary to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [13]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.416893  -0.864792  -0.279884
 -0.261323  -0.180886   0.948151
 -0.870581   0.468418  -0.15058

**Eigendecompositions**

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [14]:
Asym = A + A'

AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.6845850302713503
 -0.21926352837677915
  3.018278221798017
vectors:
3×3 Matrix{Float64}:
 -0.578456   0.524778  -0.624497
 -0.195604  -0.832484  -0.518371
  0.791914   0.177701  -0.584204

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g., that $A^{-1} = (VAV^{-1})^{-1} = VA^{-1}V^{-1}$

In [15]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          9.4369e-16  2.77556e-16
 -4.44089e-16  1.0         1.11022e-16
  2.22045e-15  1.9984e-15  1.0

# Special matrix structures

Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system.

In [16]:
n=1000
A = randn(n,n)

1000×1000 Matrix{Float64}:
 -0.170173    0.449764   0.418922   …   0.0765621  -0.608347   0.927496
 -0.568386   -1.02332    0.268471      -0.900554   -0.728072  -0.456117
 -1.35838     1.02273   -0.709727       0.870261   -1.30944    1.68497
 -1.23079     0.26268    0.274005      -2.44682    -0.675794  -0.861997
 -2.49219     1.20794    0.60891        1.42312     0.243537   0.769205
 -1.17777     1.00541   -0.60207    …   1.303       0.900621   0.0110131
  0.610323   -0.313819   0.272685       0.967454    1.63768    0.241092
 -0.476148   -0.222604   0.343215       0.485629   -0.593052   1.28417
 -1.35716    -0.390429  -0.0238847      1.61431    -2.17262   -1.32605
 -0.937341   -0.250715  -0.73475        1.83622    -0.481781   0.358581
  ⋮                                 ⋱                         
 -0.373309    0.782192   0.502226       1.47237     0.964867  -0.483396
  1.46015    -0.113071   0.0741491      1.49649     0.32096   -1.39151
  1.55036     0.782345  -1.35598       -0.567031 

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

3.1190238384387605

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal`, and `SymTridiagonal`.

In [26]:
Asym_explicit = Symmetric(Asym_noisy);

Let us compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.396973 seconds (133.94 k allocations: 15.454 MiB, 49.50% compilation time)


In [24]:
@time eigvals(Asym_noisy);

  1.007631 seconds (13 allocations: 7.920 MiB, 8.54% gc time)

In [27]:
@time eigvals(Asym_explicit);

  0.237145 seconds (5.88 k allocations: 8.358 MiB, 2.75% compilation time)

In this example, using Symmetric() on Asym_noisy made our calculations about 5x more efficient :)

## A big problem

Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [28]:
n=1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.904051 seconds (829.33 k allocations: 229.703 MiB, 10.36% gc time, 38.24% compilation time)


6.486845320895192

# Generic linear algebra

The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also support generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

**Rational numbers**

Julia has rational numbers built-in. To construct a rational number, use double forward slashes:

In [29]:
1//2

1//2

**Example: Rational linear system of equations**

The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`.

In [30]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3,3))/10

3×3 Matrix{Rational{BigInt}}:
 4//5  3//10  2//5
 4//5  1//1   1//2
 1//1  1//2   1//10

In [31]:
x = fill(1,3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  3//2
 23//10
  8//5

In [32]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [33]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1  0//1
 4//5   1//1  0//1
 4//5  -1//6  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  1//2   1//10
 0//1  3//5  21//50
 0//1  0//1  39//100

# Exercises

**11.1**

What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

and assign it a variable A_eigv

In [34]:
using LinearAlgebra

In [47]:

A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]


5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [36]:
A_eigv = eigvals(A)

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [37]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

**11.2**

Create a `Diagonal` matrix from the eigenvalues of A.

In [46]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [ ]:
@assert A_diag ==  [ -128.493     ⋅        ⋅        ⋅         ⋅ 
⋅     -55.8878    ⋅        ⋅         ⋅ 
⋅        ⋅      42.7522    ⋅         ⋅ 
⋅        ⋅        ⋅      87.1611     ⋅ 
⋅        ⋅        ⋅        ⋅      542.468]

**11.3**

Create a `LowerTriangular` matrix from A and store it in `A_lowertri`

In [48]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36